In [5]:
# import des librairies
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
import numpy as np 

In [3]:
# import des données
df = pd.read_csv("../data/silver/silver_data.csv")
df.head(3)

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
0,Data Engineer sénior (F/H) CDI (H/F),"En tant que Data Engineer chez Quantmetry, vou...",75 - PARIS 08,Actualisé le 12 juillet 2023,38H Travail en journée,40000.0 - 50000.0,Contrat à durée indéterminée,5 ans,NaN,"['Concevoir et gérer un projet', ""Concevoir un...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
1,Data Consultant Stratégie Sénior (H/F),Nous recrutons des personnes avec une appétenc...,75 - PARIS 08,Actualisé le 12 juillet 2023,35H Travail en journée,3100.0 - 5000.0,Contrat à durée indéterminée,5 ans,NaN,"[""Analyser les résultats d'un projet"", ""Déclin...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
2,Chef de projets Performance Durable/Energie/Da...,Intégré(e) au sein de la Direction Performance...,92 - ASNIERES SUR SEINE,Actualisé le 13 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,3 ans,NaN,['Contrôler et faire appliquer le respect de d...,Employé qualifié,Activités des sièges sociaux,NEXITY,250 à 499 salariés,Nexity est aujourd hui leader sur les différen...,tableau,92,ASNIERES SUR SEINE,2023/07/13


In [4]:
df.isnull().sum()

jobs                      0
description               0
loc                     902
datePosted              902
workhours              1542
salary                 1591
contract_type             0
experience                0
diploma                1827
skills                  861
qualifications         1072
industry               1031
company                 965
size                    706
company_description     947
tools                   930
ID_dep                    0
ville                   228
date                      0
dtype: int64

### preprocessing

In [11]:
df = df[df['company_description'].notna()]
df.shape

(954, 19)

In [26]:
company_description = df["company_description"].tolist()
text = company_description[20]
text

"Fondée en 2013 par Mickael, TRIBEKAI (la tribu de la VICTOIRE) est devenu une référence en cadrage stratégique, développement de franchise, d'immobilier et International.C'est pourquoi KAPP RETAIL est né en 2023 pour se mettre au service de TRIBEKAI et pour ne plus jamais dépendre de fausses solutions périmées, chronophages et coûteuses pour nos clients.\xa0Pour changer le monde et atteindre rapidement un avenir collectif meilleur, nous avons réalisé 1 levée de f..."

### Models

In [14]:
# initialize the model architecture and weights
model = T5ForConditionalGeneration.from_pretrained("t5-base")
# initialize the model tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

/home/selmane/.pyenv/versions/3.8.12/envs/transformers_summarization/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We 

In [28]:
# encode the text into tensor of integers using the appropriate tokenizer
inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=256, truncation=True)

In [29]:
# generate the summarization output
outputs = model.generate(
    inputs, 
    max_length=150, 
    min_length=40, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True)
# just for debugging
print(outputs)
print(tokenizer.decode(outputs[0]))

tensor([[    0,     3, 16840, 10059, 12048,   196,    41,   521,     3,  5135,
            20,    50,  9766,   254,  5647, 14132,    61,   259,     3, 20830,
           245, 12799,     3,    35,   212,    26,  4843, 28840,     6,  6909,
            20,  7884,     6,     3,    26,    31, 19194,     3,    15,    17,
          1331,     5,   480, 13747,     3, 27514, 22862,   259, 14563,     3,
            35,   460,  2773,   171,   142,     3,  4412,   185,   313,    20,
             3, 16840, 10059, 12048,   196,     3,     5,     1]])
<pad> TRIBEKAI (la tribu de la VICTOIRE) est devenu une référence en cadrage stratégique, développement de franchise, d'immobilier et International. KAPP RETAIL est né en 2023 pour se mettre au service de TRIBEKAI.</s>
